In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_order=pd.read_excel(r"D:\数模\第四次模拟题——在线零售商的订单履行\附件一.xlsx")
raw_warehouse=pd.read_excel(r"D:\数模\第四次模拟题——在线零售商的订单履行\附件二.xlsx")

In [3]:

SKU1=raw_order['订单中的SKU']
SKU_in_order=[]
# 原有数据类型为'[]'型，先进行转化,最终要得到['','']的形式
for order in SKU1:
    if type(order)==int:
        SKU_in_order.append(str(order))
    else:
        order=order[1:-1].split(",")
        SKU_in_order.append(order)



In [4]:
SKU_w1=raw_warehouse['仓库1'][:-1]
pos_w1=[0.516205320509211,83.7701245337600]
SKU_w2=raw_warehouse['仓库2'][:-1]
pos_w2=[30.0817907887535,1.17493342829637]
SKU_w3=raw_warehouse['仓库3'][:-1]
pos_w3=[58.8349268443702,89.8949407626723]

In [5]:
# 储存货品编号
W=[]
W1=[]
W2=[]
W3=[]
for i in range(len(SKU_w1)):
    if SKU_w1[i]==1:
        W1.append(str(i+1))
    if SKU_w2[i]==1:
        W2.append(str(i+1))
    if SKU_w3[i]==1:
        W3.append(str(i+1))
W1=set(W1)
W2=set(W2)
W3=set(W3)
W.append(W1)
W.append(W2)
W.append(W3)

In [6]:
pos_order=raw_order['订单对应的顾客位置坐标']
pos_o=[]
for i in pos_order:
    i=i[1:-1].split(",")
    pos_o.append([float(t) for t in i])


In [7]:
import math
def dis(order_pos):
    dis1=math.sqrt((order_pos[0]-pos_w1[0])**2+(order_pos[1]-pos_w1[1])**2)
    dis2=math.sqrt((order_pos[0]-pos_w2[0])**2+(order_pos[1]-pos_w2[1])**2)
    dis3=math.sqrt((order_pos[0]-pos_w3[0])**2+(order_pos[1]-pos_w3[1])**2)
    return [dis1,dis2,dis3]

In [8]:
dis(pos_o[0])

[86.38490479276503, 52.393894715563654, 57.34161048780241]

In [9]:
transfee=[
    [0,1.95,1.4],
    [1.95,0,2.1],
    [1.4,2.1,0]
]

In [34]:
cost=[]
for i in range(len(SKU_in_order)):
    order_need=set(SKU_in_order[i])
    pos=dis(pos_o[i])
    A=pos.index(min(pos))
    C=pos.index(max(pos))
    cost1=100
    cost2=100
    cost3=100
    cost4=100
    cost5=100
    cost6=100
    cost7=100
    cost8=100
    cost9=100
    cost10=100
    cost11=100
    cost12=100


    for j in pos:
        if j<max(pos) and j >min(pos):
            B=pos.index(j)

    if min(pos)<5:
            costA=3
    else :
            costA=3+0.2*(min(pos)-5)

    if pos[B]<5:
            costB=3
    else :
            costB=3+0.2*(pos[B]-5)
    
    if max(pos)<5:
            costC=3
    else :
            costC=3+0.2*(max(pos)-5)

    # 仅需A就可以满足
    if order_need.issubset(W[A]):
        cost.append(costA)
        continue
    # 从其他两个仓库直接发出
    if order_need.issubset(W[B]):
        cost1=costB
    if order_need.issubset(W[C]):
        cost2=costC


    # AB
    if order_need.issubset(W[B]|W[A]):
        trans_fee1=len(order_need.difference(W[A]))*transfee[A][B] #从A去的转运费
        trans_fee2=len(order_need.difference(W[B]))*transfee[A][B] #从B去的转运费
        cost3=min(costA+trans_fee1,costA+costB,costB+trans_fee2)
        #continue
    # AC
    if order_need.issubset(W[C]|W[A]):
        trans_fee1=len(order_need.difference(W[A]))*transfee[A][C] #从A去的转运费
        trans_fee2=len(order_need.difference(W[C]))*transfee[A][C] #从C去的转运费
        cost4=min(costA+trans_fee1,costA+costC,costC+trans_fee2)
        #continue
    # BC
    if order_need.issubset(W[B]|W[C]):
        cost5=min(costB+costC,costB+len(order_need.difference(W[B]))*transfee[B][C],costC+len(order_need.difference(W[C]))*transfee[B][C])


    # 必须要三个仓库

    # 都直接去
    cost6=costA+costB+costC


    # B,C都转到A,统一由A去终点
    # 需要决定B,C转多少货品去A
    # 如果B去A的运费低，则C只运B没有的
    n_sat=order_need.difference(W[A])
    n_sat_2=n_sat.difference(W[B]) # A也没有，B也没有,必须由C发

    if transfee[B][A]<=transfee[C][A]:
        cost7=len(n_sat_2)*transfee[C][A]+(len(n_sat)-len(n_sat_2))*transfee[B][A]+costA
    else:
        n_sat_3=n_sat.difference(W[C])
        cost7=(len(n_sat)-len(n_sat_3))*transfee[C][A]+len(n_sat_3)*transfee[B][A]+costA

    # A,B发货
    # 比较C->B和C->A的转运费
    # C->A  A,B发货
    if transfee[C][B]>transfee[C][A]:
        cost8=transfee[C][A]*len(n_sat_2)+costA+costB
    # C->B  A,B发货
    else:
        cost8=transfee[C][B]*len(n_sat_2)+costA+costB
    
    
    # AC发货
    # 比较B->A和B->C的转运费
    # C->客户 B->A
    if transfee[B][A]<=transfee[C][A]:
        cost9=costC+len(order_need.difference(W[C]|W[A]))*transfee[B][A]+costA
    # C->客户 B->C
    else:
        cost9=costC+len(order_need.difference(W[C]|W[A]))*transfee[C][B]+costA

    # BC发货
    # 比较A->B和A->C的转运费
    if transfee[A][B]<=transfee[A][C]:
        cost10=costC+len(order_need.difference(W[C]|W[B]))*transfee[B][A]+costB
    # C->客户 B->C
    else:
        cost10=costC+len(order_need.difference(W[C]|W[B]))*transfee[A][C]+costB    

    # C,A都转到B
    if transfee[C][B]>=transfee[A][B]:
        cost11=costB+transfee[C][B]*len(order_need.difference(W[A]|W[B]))+transfee[A][B]*(len(order_need.difference(W[B]))-len(order_need.difference(W[A]|W[B])))
    else:
        cost11=costB+transfee[A][B]*len(order_need.difference(W[C]|W[B]))+transfee[C][B]*(len(order_need.difference(W[B]))-len(order_need.difference(W[C]|W[B])))

    # A,B都转到C
    if transfee[A][C]>=transfee[B][C]:
        cost11=costB+transfee[A][C]*len(order_need.difference(W[C]|W[B]))+transfee[B][C]*(len(order_need.difference(W[C]))-len(order_need.difference(W[C]|W[B])))
    else:
        cost11=costB+transfee[B][C]*len(order_need.difference(W[C]|W[A]))+transfee[A][C]*(len(order_need.difference(W[C]))-len(order_need.difference(W[C]|W[A])))
    cost.append(min(cost1,cost2,cost3,cost4,cost5,cost6,cost7,cost8,cost9,cost10,cost11,cost12))
    
    

In [10]:
order_need=set(SKU_in_order[0])
pos=dis(pos_o[0])
print(3+0.2*(pos[2]-5))
sa=order_need.difference(W[2])
print(sa)
print(sa.issubset(W[2]))

13.468322097560483
{'10', '36', '70'}
False


In [35]:
print(sum(cost))
for i in range(0,10):
    print(cost[i])

print(len(cost))

574709.3670648924
16.52877894311273
9.038227045490675
11.714438945919525
11.885450925725946
6.183430223610827
11.329215105099605
12.951271936325915
15.32074186113215
14.553459388847125
12.17620797388831
50000
